In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import re

In [7]:
train_df = pd.read_csv('/content/1. 실습용자료.txt', sep = '|', encoding = 'cp949')
test_df = pd.read_csv('/content/2. 모델개발용자료.txt', sep = '|', encoding = 'cp949')

In [8]:
temp = train_df.digit_1.value_counts()[train_df.digit_1.value_counts() > 10000].index.to_list()
train_df = train_df.query('digit_1 == @temp')

In [9]:
train_df= train_df.replace(np.nan, '', regex=True)
test_df= test_df.replace(np.nan, '', regex=True)

train_df['text'] = train_df['text_obj'] + " " + train_df['text_mthd'] + " " + train_df['text_deal']
test_df['text'] = test_df['text_obj'] + " " + test_df['text_mthd'] + " " + test_df['text_deal']

In [10]:
target_list = ['digit_1']
le = LabelEncoder()
train_df['digit_1'] = le.fit_transform(train_df['digit_1'])

In [11]:
punc = re.compile('[\!@#$%\^&\*\(\)\-\=\[\]\{\}\.,/\?~\+\'"|_:;><`┃]')

def remove_punctuation(text, lower = True):
    text = punc.sub(' ', str(text))
    text = ' '.join(text.split())
    if lower:
        text = text.lower()
    return text

train_df['text'] = train_df['text'].map(remove_punctuation)
test_df['text'] = test_df['text'].map(remove_punctuation)

train_df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text
0,id_0000001,10,95,952,카센터에서,자동차부분정비,타이어오일교환,카센터에서 자동차부분정비 타이어오일교환
1,id_0000002,2,47,472,상점내에서,일반인을 대상으로,채소.과일판매,상점내에서 일반인을 대상으로 채소 과일판매
2,id_0000003,2,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매,절단하여사업체에도매 공업용고무를가지고 합성고무도매
3,id_0000004,2,47,475,영업점에서,일반소비자에게,열쇠잠금장치,영업점에서 일반소비자에게 열쇠잠금장치
4,id_0000005,8,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육,어린이집 보호자의 위탁을 받아 취학전아동보육
...,...,...,...,...,...,...,...,...
405469,id_0405470,4,56,562,커피,,,커피
405470,id_0405471,0,13,132,가게에서,일반 소비자에게,"천막, 망 제조 판매",가게에서 일반 소비자에게 천막 망 제조 판매
405471,id_0405472,3,49,493,개별화물자동차로,고객의 요청으로,화물운송서비스,개별화물자동차로 고객의 요청으로 화물운송서비스
405472,id_0405473,10,96,969,철학관,,,철학관


In [12]:
data_list = []
for q, label in zip(train_df['text'], train_df['digit_1'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [13]:
data_list[0]

['카센터에서 자동차부분정비 타이어오일교환', '10']

In [14]:
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, random_state=0)

In [15]:
from torch.utils.data import Dataset, DataLoader

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [16]:
!pip install mxnet

     |████████████████████████████████| 47.3 MB 1.2 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [17]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-hawmd69l
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-hawmd69l
     |████████████████████████████████| 132 kB 8.1 MB/s 
     |████████████████████████████████| 344 kB 67.4 MB/s 
     |████████████████████████████████| 4.5 MB 32.3 MB/s 
     |████████████████████████████████| 1.2 MB 72.8 MB/s 
     |████████████████████████████████| 4.0 MB 87.3 MB/s 
     |████████████████████████████████| 77 kB 8.0 MB/s 
     |████████████████████████████████| 596 kB 71.6 MB/s 
     |████████████████████████████████| 6.5 MB 45.7 MB/s 
     |████████████████████████████████| 895 kB 64.2 MB/s 
     |████████████████████████████████| 79 kB 9.2 MB/s 
     |████████████████████████████████| 8.7 MB 47.8 MB/s 
     |████████████████████████████████| 127 kB 74.1 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15674 sha256=fc76a6399ff10

In [18]:
from kobert.pytorch_kobert import get_pytorch_kobert_model

max_len = 32
batch_size = 32
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [19]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
import gluonnlp as nlp

tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_val = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [20]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_val, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [21]:
device = torch.device("cuda:0")

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes= len(train_df.digit_1.value_counts()),   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [22]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel


model = BERTClassifier(bertmodel,  dr_rate=0.4).to(device)
 
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# class weights balancing
class_weights = []

for i in train_df.digit_1.unique():
    weights = (1-len(train_df[train_df.digit_1 == i])/len(train_df))
    class_weights.append(weights)

class_weights_t = torch.tensor(class_weights).cuda()

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss(weight = class_weights_t) # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
from tqdm import tqdm, tqdm_notebook

train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} texst acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


  0%|          | 0/9663 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.4417967796325684 train acc 0.03125
epoch 1 batch id 201 loss 2.1104230880737305 train acc 0.13743781094527363
epoch 1 batch id 401 loss 2.035764455795288 train acc 0.220464463840399
epoch 1 batch id 601 loss 1.2723798751831055 train acc 0.32705906821963393
epoch 1 batch id 801 loss 0.9655675292015076 train acc 0.4378901373283396
epoch 1 batch id 1001 loss 0.38749703764915466 train acc 0.5233516483516484
epoch 1 batch id 1201 loss 0.43350014090538025 train acc 0.5882858034970858
epoch 1 batch id 1401 loss 0.2934722602367401 train acc 0.6383832976445396
epoch 1 batch id 1601 loss 0.2568449079990387 train acc 0.6770768269831355
epoch 1 batch id 1801 loss 0.2077726572751999 train acc 0.7079747362576346
epoch 1 batch id 2001 loss 0.12227615714073181 train acc 0.7328991754122939
epoch 1 batch id 2201 loss 0.27439039945602417 train acc 0.7535779191276692
epoch 1 batch id 2401 loss 0.29047998785972595 train acc 0.7705383173677635
epoch 1 batch id 2601 loss 0.284866839

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 2 batch id 1 loss 0.06336075812578201 train acc 0.96875
epoch 2 batch id 201 loss 0.1943783462047577 train acc 0.9692164179104478
epoch 2 batch id 401 loss 0.09370286762714386 train acc 0.9680486284289277
epoch 2 batch id 601 loss 0.045317329466342926 train acc 0.9683340266222962
epoch 2 batch id 801 loss 0.013426819816231728 train acc 0.9691401373283396
epoch 2 batch id 1001 loss 0.006645128130912781 train acc 0.9690934065934066
epoch 2 batch id 1201 loss 0.23824045062065125 train acc 0.9695826394671108
epoch 2 batch id 1401 loss 0.01161443442106247 train acc 0.9696422198429693
epoch 2 batch id 1601 loss 0.30915528535842896 train acc 0.9696869144284822
epoch 2 batch id 1801 loss 0.12483498454093933 train acc 0.969981954469739
epoch 2 batch id 2001 loss 0.02365957945585251 train acc 0.9701867816091954
epoch 2 batch id 2201 loss 0.2123349905014038 train acc 0.9703401862789641
epoch 2 batch id 2401 loss 0.2446490079164505 train acc 0.9704289879216993
epoch 2 batch id 2601 loss 0.16

  0%|          | 0/2416 [00:00<?, ?it/s]

epoch 2 texst acc 0.9760839197019867


  0%|          | 0/9663 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 0.09041742980480194 train acc 0.96875
epoch 3 batch id 201 loss 0.15458887815475464 train acc 0.9763681592039801
epoch 3 batch id 401 loss 0.005783995147794485 train acc 0.9771664588528678
epoch 3 batch id 601 loss 0.023920340463519096 train acc 0.9769134775374376
epoch 3 batch id 801 loss 0.022351225838065147 train acc 0.9771769662921348
epoch 3 batch id 1001 loss 0.004780394956469536 train acc 0.9770541958041958
epoch 3 batch id 1201 loss 0.1575746089220047 train acc 0.9773105745212323
epoch 3 batch id 1401 loss 0.24324995279312134 train acc 0.9775606709493219
epoch 3 batch id 1601 loss 0.1188083216547966 train acc 0.977396939412867
epoch 3 batch id 1801 loss 0.08629803359508514 train acc 0.9776339533592449
epoch 3 batch id 2001 loss 0.2500670552253723 train acc 0.9777611194402799
epoch 3 batch id 2201 loss 0.24845066666603088 train acc 0.9776238073602908
epoch 3 batch id 2401 loss 0.1972748190164566 train acc 0.9775744481466055
epoch 3 batch id 2601 loss 0.19

In [ ]:
test_data = test_df[['text']]
test_label = [0 for i in range(len(test_df))]
test_data['label'] = test_label

test_data = [list(test_data.iloc[i]) for i in range(len(test_data))]
print(test_data[:10])
test_data

In [ ]:
data_test = BERTDataset(test_data, 0, 1, tok, max_len, True, False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [ ]:
predictions =[]
with torch.no_grad():
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().cuda()
    segment_ids = segment_ids.long().cuda()
    valid_length= valid_length
    label = label.long().cuda()
    out = model(token_ids, valid_length, segment_ids)
    _, max_indices = torch.max(out, 1)

    predictions.extend(max_indices.squeeze(0).detach().cpu().numpy())


In [ ]:
final = le.inverse_transform(predictions)

In [ ]:
test_df['digit_1'] = final
test_df

In [ ]:
test_df.to_csv('E10_Reduction_digit1.csv', index = False)

In [ ]:
predictions =[]
with torch.no_grad():
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().cuda()
    segment_ids = segment_ids.long().cuda()
    valid_length= valid_length
    label = label.long().cuda()
    out = model(token_ids, valid_length, segment_ids)
    max_indices = torch.nn.functional.softmax(out, dim=1).detach().cpu().numpy()

    predictions.extend(max_indices)

In [ ]:
x = np.array(predictions)

In [ ]:
np.save('Reduction_E10_digit1', x)